## 1_Tester un modèle BERT Tiny est une excellente idée pour réduire le temps d'entraînement et utiliser moins de ressources tout en explorant les capacités du modèle :

In [20]:
# Import des bibliothèques
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm

# Charger le tokenizer TinyBERT
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

# Préparer les données pour TinyBERT
def preprocess_data(texts, labels, tokenizer, max_length=128):
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )
    labels = torch.tensor(labels.tolist())
    return encodings, labels

# Charger les datasets
train_data = pd.read_csv("C:/Users/chume/Projet_Final/data/processed/train_cleaned_final.csv")
test_data = pd.read_csv("C:/Users/chume/Projet_Final/data/processed/test_cleaned.csv")

# Vérifiez les colonnes disponibles
print(train_data.head())
print(test_data.head())

# Séparer les données en train/validation
X_train, X_val, y_train, y_val = train_test_split(
    train_data['text'], train_data['label'], test_size=0.2, random_state=42
)

# Tokenisation et encodage des données
train_encodings, train_labels = preprocess_data(X_train, y_train, tokenizer)
val_encodings, val_labels = preprocess_data(X_val, y_val, tokenizer)
test_encodings, test_labels = preprocess_data(test_data['text'], test_data['label'], tokenizer)

# Préparer les DataLoaders
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Charger le modèle TinyBERT
model = BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimiseur
optimizer = AdamW(model.parameters(), lr=5e-5)

# Entraînement
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # Afficher la progression
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

# Évaluation sur le jeu de validation
model.eval()
val_predictions = []
val_labels_list = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        val_predictions.extend(predictions.cpu().numpy())
        val_labels_list.extend(labels.cpu().numpy())

# Rapport de classification sur le jeu de validation
print("Validation Metrics:")
print(classification_report(val_labels_list, val_predictions, target_names=["Classe 0", "Classe 1"]))

# Évaluation sur le jeu de test
test_predictions = []
test_labels_list = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels_list.extend(labels.cpu().numpy())

# Rapport de classification sur le jeu de test
print("Test Metrics:")
print(classification_report(test_labels_list, test_predictions, target_names=["Classe 0", "Classe 1"]))


  id                                              title  \
0  0  Palestinians switch off Christmas lights in Be...   
1  1  China says Trump call with Taiwan president wo...   
2  2   FAIL! The Trump Organization’s Credit Score W...   
3  3  Zimbabwe military chief's China trip was norma...   
4  4  THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...   

                                                text  label  text_length  
0  RAMALLAH, West Bank (Reuters) - Palestinians s...      1         1335  
1  BEIJING (Reuters) - U.S. President-elect Donal...      1          373  
2  While the controversy over Trump s personal ta...      0         2072  
3  BEIJING (Reuters) - A trip to Beijing last wee...      1         2692  
4  There has never been a more UNCOURAGEOUS perso...      0         1946  
   id                                              title  \
0   0  Live from New York, it's a Trump-Clinton remat...   
1   1  Catalan separatists to lose majority in tight ...   
2   2  North Ca

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\chume\.conda\envs\llama_env\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


Epoch 1: 100%|██████████| 1218/1218 [00:43<00:00, 27.78it/s, loss=0.00184]


Epoch 2/3


Epoch 2: 100%|██████████| 1218/1218 [00:43<00:00, 27.73it/s, loss=0.000765]


Epoch 3/3


Epoch 3: 100%|██████████| 1218/1218 [00:43<00:00, 27.70it/s, loss=0.000262]


Validation Metrics:
              precision    recall  f1-score   support

    Classe 0       0.99      0.98      0.99      2208
    Classe 1       0.99      0.99      0.99      2662

    accuracy                           0.99      4870
   macro avg       0.99      0.99      0.99      4870
weighted avg       0.99      0.99      0.99      4870

Test Metrics:
              precision    recall  f1-score   support

    Classe 0       0.99      0.98      0.99      3750
    Classe 1       0.99      0.99      0.99      4363

    accuracy                           0.99      8113
   macro avg       0.99      0.99      0.99      8113
weighted avg       0.99      0.99      0.99      8113



## Analyse des résultats pour TinyBERT

1. **Validation Metrics** :
   - **Classe 0** :
     - **Précision** : 99 % (quasi toutes les prédictions pour cette classe sont correctes).
     - **Rappel** : 98 % (le modèle a détecté 98 % des exemples réels de cette classe).
     - **F1-score** : 99 % (équilibre entre précision et rappel).
   - **Classe 1** :
     - Tous les scores sont proches de 99 %, ce qui indique une excellente performance.
   - **Précision globale** : 99 % sur le jeu de validation (4870 exemples).

2. **Test Metrics** :
   - **Classe 0 et Classe 1** :
     - Tous les scores (précision, rappel, F1-score) sont autour de 99 %.
   - **Précision globale** : 99 % sur le jeu de test (8113 exemples).

3. **Moyennes pondérées et globales** :
   - **Macro avg** et **Weighted avg** à 99 % confirment que les deux classes sont bien équilibrées dans leurs performances.

---

## Points positifs :
- **Précision élevée** : Indique que TinyBERT gère très bien votre jeu de données.
- **Équilibre entre les classes** : Les métriques pour les deux classes sont quasiment identiques, ce qui suggère que le modèle n'est pas biaisé vers une classe particulière.
- **Généralisation solide** : Les performances sur le jeu de validation et de test sont similaires, ce qui montre une bonne généralisation du modèle.

---


## 2_Pour tester et évaluer un modèle BERT Small, le processus est similaire à celui de TinyBERT. Hugging Face propose des modèles BERT Small, souvent appelés "distilled" ou "light". Voici comment procéder.

In [21]:
# Import des bibliothèques
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm

# Charger le tokenizer BERT Small
model_name = "prajjwal1/bert-small"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Préparer les données pour BERT Small
def preprocess_data(texts, labels, tokenizer, max_length=128):
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )
    labels = torch.tensor(labels.tolist())
    return encodings, labels

# Charger les datasets
train_data = pd.read_csv("C:/Users/chume/Projet_Final/data/processed/train_cleaned_final.csv")
test_data = pd.read_csv("C:/Users/chume/Projet_Final/data/processed/test_cleaned.csv")

# Vérifiez les colonnes disponibles
print(train_data.head())
print(test_data.head())

# Séparer les données en train/validation
X_train, X_val, y_train, y_val = train_test_split(
    train_data['text'], train_data['label'], test_size=0.2, random_state=42
)

# Tokenisation et encodage des données
train_encodings, train_labels = preprocess_data(X_train, y_train, tokenizer)
val_encodings, val_labels = preprocess_data(X_val, y_val, tokenizer)
test_encodings, test_labels = preprocess_data(test_data['text'], test_data['label'], tokenizer)

# Préparer les DataLoaders
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Charger le modèle BERT Small
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimiseur
optimizer = AdamW(model.parameters(), lr=5e-5)

# Entraînement
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # Afficher la progression
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

# Évaluation sur le jeu de validation
model.eval()
val_predictions = []
val_labels_list = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        val_predictions.extend(predictions.cpu().numpy())
        val_labels_list.extend(labels.cpu().numpy())

# Rapport de classification sur le jeu de validation
print("Validation Metrics:")
print(classification_report(val_labels_list, val_predictions, target_names=["Classe 0", "Classe 1"]))

# Évaluation sur le jeu de test
test_predictions = []
test_labels_list = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels_list.extend(labels.cpu().numpy())

# Rapport de classification sur le jeu de test
print("Test Metrics:")
print(classification_report(test_labels_list, test_predictions, target_names=["Classe 0", "Classe 1"]))


  id                                              title  \
0  0  Palestinians switch off Christmas lights in Be...   
1  1  China says Trump call with Taiwan president wo...   
2  2   FAIL! The Trump Organization’s Credit Score W...   
3  3  Zimbabwe military chief's China trip was norma...   
4  4  THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...   

                                                text  label  text_length  
0  RAMALLAH, West Bank (Reuters) - Palestinians s...      1         1335  
1  BEIJING (Reuters) - U.S. President-elect Donal...      1          373  
2  While the controversy over Trump s personal ta...      0         2072  
3  BEIJING (Reuters) - A trip to Beijing last wee...      1         2692  
4  There has never been a more UNCOURAGEOUS perso...      0         1946  
   id                                              title  \
0   0  Live from New York, it's a Trump-Clinton remat...   
1   1  Catalan separatists to lose majority in tight ...   
2   2  North Ca

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\chume\.conda\envs\llama_env\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


Epoch 1: 100%|██████████| 1218/1218 [01:06<00:00, 18.23it/s, loss=0.000439]


Epoch 2/3


Epoch 2: 100%|██████████| 1218/1218 [01:07<00:00, 18.16it/s, loss=0.000181]


Epoch 3/3


Epoch 3: 100%|██████████| 1218/1218 [01:07<00:00, 18.16it/s, loss=0.00063] 


Validation Metrics:
              precision    recall  f1-score   support

    Classe 0       0.99      0.98      0.99      2208
    Classe 1       0.98      0.99      0.99      2662

    accuracy                           0.99      4870
   macro avg       0.99      0.99      0.99      4870
weighted avg       0.99      0.99      0.99      4870

Test Metrics:
              precision    recall  f1-score   support

    Classe 0       1.00      0.98      0.99      3750
    Classe 1       0.98      1.00      0.99      4363

    accuracy                           0.99      8113
   macro avg       0.99      0.99      0.99      8113
weighted avg       0.99      0.99      0.99      8113



## Analyse des résultats pour **BERT Small**

### **1. Validation Metrics**
- **Classe 0 (support : 2208)** :
  - **Précision** : 99 % (quasiment toutes les prédictions positives pour cette classe sont correctes).
  - **Rappel** : 98 % (le modèle a identifié 98 % des vrais exemples de cette classe).
  - **F1-score** : 99 %, ce qui indique un excellent équilibre entre précision et rappel.
- **Classe 1 (support : 2662)** :
  - **Précision** : 98 % (légèrement inférieure à celle de la Classe 0, mais reste excellente).
  - **Rappel** : 99 % (presque tous les exemples réels de cette classe ont été détectés).
- **Précision globale** : **99 %** sur le jeu de validation, montrant une très bonne généralisation.

### **2. Test Metrics**
- **Classe 0 (support : 3750)** :
  - **Précision** : 100 % (toutes les prédictions pour cette classe sont exactes).
  - **Rappel** : 98 % (quelques exemples de la Classe 0 n’ont pas été détectés).
  - **F1-score** : 99 %, identique à la validation.
- **Classe 1 (support : 4363)** :
  - **Précision** : 98 %, cohérent avec les résultats sur le jeu de validation.
  - **Rappel** : 100 %, aucun exemple de la Classe 1 n’a été manqué.
- **Précision globale** : **99 %**, montrant une excellente performance sur le jeu de test.

---

## Comparaison avec TinyBERT

| Modèle          | Validation Accuracy | Test Accuracy | Macro Avg Precision | Macro Avg Recall | Macro Avg F1-Score |
|------------------|---------------------|---------------|---------------------|------------------|---------------------|
| **TinyBERT**    | 99 %               | 99 %          | 99 %               | 99 %            | 99 %               |
| **BERT Small**  | 99 %               | 99 %          | 99 %               | 99 %            | 99 %               |

Les résultats de BERT Small et TinyBERT sont similaires, ce qui est courant lorsque le dataset est bien équilibré et de taille modérée. Cependant :
- BERT Small a une capacité légèrement supérieure, ce qui peut le rendre plus performant sur des tâches ou datasets plus complexes.
- TinyBERT est plus rapide et léger, ce qui est avantageux pour les systèmes avec des ressources limitées.

---

## Recommandations
1. **Choix du modèle** :
   - Si les performances actuelles suffisent pour votre projet, **TinyBERT** est un excellent choix grâce à son efficacité.
   - Si vous prévoyez de traiter des datasets plus complexes, **BERT Small** pourrait offrir un avantage.

2. **Prochaines étapes** :
   - **Déploiement** : Envisagez de déployer le modèle via une API ou une application web (par exemple, Flask, FastAPI).
   - **Évaluation en conditions réelles** : Testez le modèle sur des données réelles pour vérifier sa robustesse.
   - **Fine-tuning sur d'autres tâches** : Si vous souhaitez explorer d’autres tâches (comme la classification multi-label ou la détection d’entités nommées), BERT Small pourrait être un bon point de départ.


# 3_Lazy Predict

Lazy Predict est une bibliothèque Python qui permet de tester rapidement plusieurs modèles de machine learning (ML) sur un dataset sans devoir coder chaque pipeline individuellement. Elle est utile pour :

Obtenir une vue d'ensemble des performances de différents modèles ML.
Comparer les modèles pour un problème spécifique.

In [22]:
!pip install lazypredict


   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 73.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ----------------- --------------------- 56.1/124.9 MB 275.1 MB/s eta 0:00:01
   ---------------------------------- --- 114.3/124.9 MB 270.2 MB/s eta 0:00:01
   -------------------------------------- 124.9/124.9 MB 221.8 MB/s eta 0:00:00


In [27]:
import torch

# Vérifier si CUDA est disponible
print("GPU disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU détecté")


GPU disponible : True
Nom du GPU : NVIDIA GeForce GTX 1080


In [29]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
import torch
import os

# Vérifier si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Appareil utilisé :", device)

# Charger les fichiers
train_file = "C:/Users/chume/Projet_Final/data/processed/train_cleaned_final.csv"
test_file = "C:/Users/chume/Projet_Final/data/processed/test_cleaned.csv"

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

# Vérifiez les colonnes
print("Colonnes dans le fichier d'entraînement :", train_data.columns)
print("Colonnes dans le fichier de test :", test_data.columns)

# Assurez-vous d'utiliser les bonnes colonnes pour les textes et les labels
X = train_data['text']
y = train_data['label']

# Vectorisation des données avec TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000)  # Limitez à 10 000 pour la mémoire GPU
X_vec = vectorizer.fit_transform(X)

# Réduction de dimension avec TruncatedSVD
svd = TruncatedSVD(n_components=300, random_state=42)  # Réduit à 300 dimensions
X_reduced = svd.fit_transform(X_vec)

# Division des données en train/test
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Utilisation de Lazy Predict
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Afficher les résultats
print("Performances des modèles Lazy Predict :")
print(models)


Appareil utilisé : cuda
Colonnes dans le fichier d'entraînement : Index(['id', 'title', 'text', 'label', 'text_length'], dtype='object')
Colonnes dans le fichier de test : Index(['id', 'title', 'text', 'label'], dtype='object')


  3%|▎         | 1/32 [01:23<42:53, 83.03s/it]


KeyboardInterrupt: 